# Kelvin-Helmholtz Instability

<img src="http://www.flame.org/~cdoswell/SuptorRoles/Vortex%20Sheet%20Model%20copy.jpg" width="254" height="300" />

(image: Chuck Doswell)

We will simulate the incompressible Kelvin-Helmholtz problem.  We non-dimensionalize the problem by taking the box height to be one and the jump in velocity to be one.  Then the Reynolds number is given by

$ {\rm Re} = \frac{U H}{\nu} = \frac{1}{\nu}. $

We use no slip boundary conditions, and a box with aspect ratio $L/H=2$.  The initial velocity profile is given by a hyperbolic tangent, and only a single mode is initially excited.  We will also track a passive scalar which will help us visualize the instability.

First, we import the necessary modules.

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
from matplotlib import animation

In [ ]:
from dedalus import public as de
from dedalus.extras import flow_tools

Here, we set logging to `INFO` level. Currently, by default, Dedalus sets its logging output to `DEBUG`, which produces more info than we need here.

In [ ]:
import logging
root = logging.root
for h in root.handlers:
    h.setLevel("INFO")
    
logger = logging.getLogger(__name__)

## Problem Domain

First, we will specify the domain.  Domains are built by taking the direct product of bases.  Here we are running a 2D simulation, so we will define $x$ and $y$ bases.  From these, we build the domain.

In [ ]:
#Aspect ratio 2
Lx, Ly = (2., 1.)
nx, ny = (1024, 512)

# Create bases and domain
x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)
y_basis = de.Chebyshev('y',ny, interval=(-Ly/2, Ly/2), dealias=3/2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

## Equations

Next we will define the equations that will be solved on this domain.  The equations are

$$ \partial_t u + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} u + \partial_x p = \frac{1}{{\rm Re}} \nabla^2 u $$
$$ \partial_t v + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} v + \partial_y p = \frac{1}{{\rm Re}} \nabla^2 v $$
$$ \boldsymbol{\nabla}\boldsymbol{\cdot}\boldsymbol{u} = 0 $$
$$ \partial_t s + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} s = \frac{1}{{\rm Re}{\rm Sc}} \nabla^2 s $$

The equations are written such that the left-hand side (LHS) is treated implicitly, and the right-hand side (RHS) is treated explicitly.  The LHS is limited to only linear terms, though linear terms can also be placed on the RHS.  Since $y$ is our special direction in this example, we also restrict the LHS to be at most first order in derivatives with respect to $y$.

We also set the parameters, the Reynolds and Schmidt numbers.

In [ ]:
problem = de.IVP(domain, variables=['p','u','v','uy','vy','s','Sy'])

Reynolds = 1e4
Schmidt = 1.

problem.parameters['Re'] = Reynolds
problem.parameters['Sc'] = Schmidt

problem.add_equation("dt(u) + dx(p) - 1/Re*(dx(dx(u)) + dy(uy)) = - u*dx(u) - v*uy")
problem.add_equation("dt(v) + dy(p) - 1/Re*(dx(dx(v)) + dy(vy)) = - u*dx(v) - v*vy")
problem.add_equation("dx(u) + vy = 0")

problem.add_equation("dt(s) - 1/(Re*Sc)*(dx(dx(s)) + dy(Sy)) = - u*dx(s) - v*Sy")

problem.add_equation("uy - dy(u) = 0")
problem.add_equation("vy - dy(v) = 0")
problem.add_equation("Sy - dy(s) = 0")

In [ ]:
problem.add_bc("left(u) = 0.5")
problem.add_bc("right(u) = -0.5")
problem.add_bc("left(v) = 0")
problem.add_bc("right(v) = 0", condition="(nx != 0)")
problem.add_bc("integ(p,'y') = 0", condition="(nx == 0)")
problem.add_bc("left(s) = 0")
problem.add_bc("right(s) = 1")

Note that we have a special boundary condition for the $k_x=0$ mode (singled out by `condition="(dx==0)"`).  This is because the continuity equation implies $\partial_y v=0$ if $k_x=0$; thus, $v=0$ on the top and bottom are redundant boundary conditions.  We replace one of these with a gauge choice for the pressure.


## Timestepping

We have implemented about twenty implicit-explicit timesteppers in Dedalus.  This contains both multi-stage and multi-step methods.  For this problem, we will use a four-stage, fourth order Runge-Kutta integrator.  Changing the timestepping algorithm is as easy as changing one line of code.

In [ ]:
ts = de.timesteppers.RK443

## Initial Value Problem

We now have the three ingredients necessary to set up our IVP:

In [ ]:
solver =  problem.build_solver(ts)

In [ ]:
x = domain.grid(0)
y = domain.grid(1)
u = solver.state['u']
uy = solver.state['uy']
v = solver.state['v']
vy = solver.state['vy']
p = solver.state['p']
s = solver.state['s']
sy = solver.state['Sy']

a = 0.05
sigma = 0.2
flow = -0.5
u['g'] = flow*np.tanh(y/a)
s['g'] = 0.5*(1+np.tanh(y/a))

In [ ]:
amp = -0.5
sigma = 0.2
v['g'] = amp*np.exp(-y**2/sigma**2)*np.sin(2*np.pi*x/Lx)

In [ ]:
plt.close()
plt.plot(v['g'][int(nx/4),:],y[0])
plt.ylabel('y')
plt.xlabel('vertical velocity')
plt.savefig("KH_velocity.png")

Now we set integration parameters and the CFL.

In [ ]:
solver.stop_sim_time = 10.01
solver.stop_wall_time = np.inf
solver.stop_iteration = 3000

initial_dt = 0.2*Lx/nx
cfl = flow_tools.CFL(solver,initial_dt,safety=0.8,threshold=0.05)
cfl.add_velocities(('u','v'))

## Analysis

We have a sophisticated analysis framework in which the user specifies analysis tasks as strings.  Users can output full data cubes, slices, volume averages, and more.  Here we will only output a few 2D slices, and a 1D profile of the horizontally averaged concentration field.  Data is output in the hdf5 file format.

In [ ]:
analysis = solver.evaluator.add_file_handler('analysis_tasks', sim_dt=0.1, max_writes=50)
analysis.add_task('s')
analysis.add_task('u')
analysis.add_task('0.5*(u**2+v**2)',name='KE',scales=(3/2,3/2))
analysis.add_task('0.5*(dx(v)-uy)**2',name='enstrophy')
solver.evaluator.vars['Lx'] = Lx
analysis.add_task("integ(s,'x')/Lx", name='s profile')

## Main Loop

We now have everything set up for our simulation.  In Dedalus, the user writes their own main loop.
First we will run the model and save figures every n steps.
Then we will save directly in video format

In [ ]:
# Make plot of scalar field
x = domain.grid(0,scales=domain.dealias)
y = domain.grid(1,scales=domain.dealias)
xm, ym = np.meshgrid(x,y)
fig, axis = plt.subplots(figsize=(8,5))
s.set_scales(domain.dealias)
p = axis.pcolormesh(xm, ym, s['g'].T, cmap='RdBu_r');
axis.set_title('t = %f' %solver.sim_time)
axis.set_xlim([0,2.])
axis.set_ylim([-0.5,0.5])


#Ready to solve the equations
logger.info('Starting loop')
start_time = time.time()
nt = 0
while solver.ok:
    dt = cfl.compute_dt()
    solver.step(dt)
    print(f"{solver.iteration}/{solver.stop_iteration}") 
    if solver.iteration % 10 == 0:
        # Update plot of scalar field
        p.set_array(np.ravel(s['g'][:,:].T))
        axis.set_title('t = %f' %solver.sim_time)
        fig.canvas.draw()
        plt.savefig(f"./StratifiedShearedFlow_KH_instability_viscous_difusive_{nt:03d}.png")
        nt += 1
end_time = time.time()
# Print statistics
logger.info('Run time: %f' %(end_time-start_time))
logger.info('Iterations: %i' %solver.iteration)

In [ ]:




#Video

# def init():
#     axis.set_xlim([0,2.])
#     axis.set_ylim([-0.5,0.5])
#     return p,

# def update(frame):
    
#     if solver.ok:
#         dt = cfl.compute_dt()
#         solver.step(dt)
#         p.set_array(np.ravel(s['g'][:,:].T))
#         axis.set_title(f't = {solver.sim_time}')
    

# #mimation = FuncAnimation(fig, update,interval = 300)

# anim = animation.FuncAnimation(fig, update, frames=3000, interval=300)
# anim.save('KH.mp4', fps=20)

In [ ]:
# from IPython.display import HTML
# HTML(anim.to_html5_video())

## Analysis

As an example of doing some analysis, we will load in the horizontally averaged profiles of the scalar field $s$ and plot them.

In [ ]:
# # Read in the data
# f = h5py.File('analysis_tasks/analysis_tasks_s1/analysis_tasks_s1_p0.h5','r')
# y = f['/scales/y/1.0'][:]
# t = f['scales']['sim_time'][:]
# s_ave = f['tasks']['s profile'][:]
# f.close()

# s_ave = s_ave[:,0,:] # remove length-one x dimension

In [ ]:
# fig, axis = plt.subplots(figsize=(8,6))
# for i in range(0,31,6):
#   axis.plot(y,s_ave[i,:],label='t=%4.2f' %t[i])

# plt.xlim([-0.5,0.5])
# plt.ylim([0,1])
# plt.ylabel(r'$\frac{\int \ s dx}{L_x}$',fontsize=24)
# plt.xlabel(r'$y$',fontsize=24)
# plt.legend(loc='lower right').draw_frame(False)